In [ ]:
---
sidebar_position: 1
---

# Function calling

Certain chat models like [OpenAI](https://platform.openai.com/docs/guides/function-calling), [Gemini](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling), etc. have a function-calling API that lets you describe functions and their arguments, and have the model return a JSON object with a function to invoke and the inputs to that function. Function-calling is extremely useful for building [tool-using chains and agents](/docs/use_cases/tool_use/), and for getting structured outputs from models more generally.

LangChain comes with a number of utilities to make function-calling easy. Namely, it comes with:

* simple syntax for binding functions to models
* converters for formatting various types of objects to the expected function schemas
* output parsers for extracting the function invocations from API responses
* chains for getting structured outputs from a model, built on top of function calling

We'll focus here on the first two points. To see how output parsing works check out the [OpenAI Tools output parsers](/docs/modules/model_io/output_parsers/types/openai_tools) and to see the structured output chains check out the [Structured output guide](/docs/guides/structured_output).

Before getting started make sure you have `langchain-core` installed.

In [1]:
%pip install -qU langchain-core langchain-openai


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import getpass

## Binding functions

A number of models implement helper methods that will take care of formatting and binding different function-like objects to the model. Let's take a look at how we might take the following Pydantic function schema and get different models to invoke it:

In [7]:
from langchain_core.pydantic_v1 import BaseModel, Field

# Note that the docstring here are crucial, as they will be passed along 
# to the model along with the class name.
class Multiply(BaseModel):
    """Multiply two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

import Tabs from '@theme/Tabs';
import TabItem from '@theme/TabItem';
import CodeBlock from "@theme/CodeBlock";

<Tabs>
  <TabItem value="openai" label="OpenAI" default>

Set up dependencies and API keys:

In [ ]:
%pip install -qU langchain-openai

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

We can use the `ChatOpenAI.bind_tools()` method to handle converting `Multiply` to an OpenAI function and binding it to the model (i.e., passing it in each time the model is invoked). 

:::{.callout-note} 
There is a legacy `ChatOpenAI.bind_functions()` method that uses OpenAI's now-deprecated `functions` API parameter instead of their
updated `tools` parameter. Using the new `tools` parameter allows us to take advantage of the parallel function-calling capabilities of the latest models.
:::

In [11]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
llm_with_tools = llm.bind_tools([Multiply])
llm_with_tools.invoke("what's 3 * 12")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Q8ZQ97Qrj5zalugSkYMGV1Uo', 'function': {'arguments': '{"a":3,"b":12}', 'name': 'Multiply'}, 'type': 'function'}]})

We can add a tool parser to extract the tool calls from the generated message to JSON:

In [12]:
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

tool_chain = llm_with_tools | JsonOutputToolsParser()
tool_chain.invoke("what's 3 * 12")

[{'type': 'Multiply', 'args': {'a': 3, 'b': 12}}]

Or back to the original Pydantic class:

In [13]:
from langchain_core.output_parsers.openai_tools import PydanticToolsParser

tool_chain = llm_with_tools | PydanticToolsParser(tools=[Multiply])
tool_chain.invoke("what's 3 * 12")

[Multiply(a=3, b=12)]

If we wanted to force that a tool is used (and that it is used only once), we can set the `tool_choice` argument:

In [17]:
llm_with_multiply = llm.bind_tools([Multiply], tool_choice="Multiply")
llm_with_multiply.invoke("make up some numbers if you really want but I'm not forcing you")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_f3DApOzb60iYjTfOhVFhDRMI', 'function': {'arguments': '{"a":5,"b":10}', 'name': 'Multiply'}, 'type': 'function'}]})

For more see the [ChatOpenAI API reference](https://api.python.langchain.com/en/latest/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html#langchain_openai.chat_models.base.ChatOpenAI.bind_tools).

  </TabItem>
  <TabItem value="google_vertexai" label="Google Vertex AI">

Install dependencies:

In [ ]:
%pip install -qU langchain-google-vertexai

Uncomment to set up Google Cloud auth:

In [ ]:
# !gcloud auth login

We can pass Pydantic classes in as function schemas by binding the `functions` argument to our model:

In [ ]:
from langchain_google_vertexai import ChatVertexAI

llm = ChatVertexAI(model="gemini-pro", temperature=0)
llm_with_functions = llm.bind(functions=[Multiply])
llm_with_functions.invoke("what's 3 * 12")

  </TabItem>
  <TabItem value="fireworks" label="Fireworks">

Install dependencies and set API keys:

In [ ]:
%pip install -qU langchain-fireworks

In [ ]:
os.environ["FIREWORKS_API_KEY"] = getpass.getpass()

We can use the `ChatFireworks.bind_tools()` method to handle converting `Multiply` to a valid function schema and binding it to the model (i.e., passing it in each time the model is invoked). 

In [9]:
from langchain_fireworks import ChatFireworks

llm = ChatFireworks(model="accounts/fireworks/models/firefunction-v1", temperature=0)
llm_with_tools = llm.bind_tools([Multiply])
llm_with_tools.invoke("what's 3 * 12")

AIMessage(content='Three multiplied by twelve is 36.')

If our model isn't using the tool, as is the case here, we can force tool usage by specifying `tool_choice="any"` or by specifying the name of the specific tool we want used:

In [11]:
llm_with_tools = llm.bind_tools([Multiply], tool_choice="Multiply")
llm_with_tools.invoke("what's 3 * 12")

AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_qIP2bJugb67LGvc6Zhwkvfqc', 'type': 'function', 'function': {'name': 'Multiply', 'arguments': '{"a": 3, "b": 12}'}}]})

We can add a tool parser to extract the tool calls from the generated message to JSON:

In [12]:
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

tool_chain = llm_with_tools | JsonOutputToolsParser()
tool_chain.invoke("what's 3 * 12")

[{'type': 'Multiply', 'args': {'a': 3, 'b': 12}}]

Or back to the original Pydantic class:

In [13]:
from langchain_core.output_parsers.openai_tools import PydanticToolsParser

tool_chain = llm_with_tools | PydanticToolsParser(tools=[Multiply])
tool_chain.invoke("what's 3 * 12")

[Multiply(a=3, b=12)]

  </TabItem>
  <TabItem value="mistral" label="Mistral">

Install dependencies and set API keys:

In [ ]:
%pip install -qU langchain-mistralai

In [ ]:
os.environ["MISTRAL_API_KEY"] = getpass.getpass()

We can use the `ChatMistralAI.bind_tools()` method to handle converting `Multiply` to a valid function schema and binding it to the model (i.e., passing it in each time the model is invoked). 

In [14]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(model="mistral-large-latest", temperature=0)
llm_with_tools = llm.bind_tools([Multiply])
llm_with_tools.invoke("what's 3 * 12")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'null', 'type': <ToolType.function: 'function'>, 'function': {'name': 'Multiply', 'arguments': '{"a": 3, "b": 12}'}}]})

We can add a tool parser to extract the tool calls from the generated message to JSON:

In [15]:
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

tool_chain = llm_with_tools | JsonOutputToolsParser()
tool_chain.invoke("what's 3 * 12")

[{'type': 'Multiply', 'args': {'a': 3, 'b': 12}}]

Or back to the original Pydantic class:

In [16]:
from langchain_core.output_parsers.openai_tools import PydanticToolsParser

tool_chain = llm_with_tools | PydanticToolsParser(tools=[Multiply])
tool_chain.invoke("what's 3 * 12")

[Multiply(a=3, b=12)]

We can force tool usage by specifying `tool_choice="any"`:

In [18]:
llm_with_tools = llm.bind_tools([Multiply], tool_choice="any")
llm_with_tools.invoke("I don't even want you to use the tool")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'null', 'type': <ToolType.function: 'function'>, 'function': {'name': 'Multiply', 'arguments': '{"a": 5, "b": 7}'}}]})

  </TabItem>
  <TabItem value="mistral" label="Mistral">

Since TogetherAI is a drop-in replacement for OpenAI, we can just use the OpenAI integration.

Install dependencies and set API keys:

In [ ]:
%pip install -qU langchain-openai

In [ ]:
os.environ["TOGETHER_API_KEY"] = getpass.getpass()

We can use the `ChatOpenAI.bind_tools()` method to handle converting `Multiply` to a valid function schema and binding it to the model (i.e., passing it in each time the model is invoked). 

In [22]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=os.environ["TOGETHER_API_KEY"],
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
)
llm_with_tools = llm.bind_tools([Multiply])
llm_with_tools.invoke("what's 3 * 12")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_4tc61dp0478zafqe33hfriee', 'function': {'arguments': '{"a":3,"b":12}', 'name': 'Multiply'}, 'type': 'function'}]})

We can add a tool parser to extract the tool calls from the generated message to JSON:

In [23]:
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

tool_chain = llm_with_tools | JsonOutputToolsParser()
tool_chain.invoke("what's 3 * 12")

[{'type': 'Multiply', 'args': {'a': 3, 'b': 12}}]

Or back to the original Pydantic class:

In [24]:
from langchain_core.output_parsers.openai_tools import PydanticToolsParser

tool_chain = llm_with_tools | PydanticToolsParser(tools=[Multiply])
tool_chain.invoke("what's 3 * 12")

[Multiply(a=3, b=12)]

If we wanted to force that a tool is used (and that it is used only once), we can set the `tool_choice` argument:

In [25]:
llm_with_multiply = llm.bind_tools([Multiply], tool_choice="Multiply")
llm_with_multiply.invoke("make up some numbers if you really want but I'm not forcing you")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_6k6d0gr3jhqil2kqf7sgeusl', 'function': {'arguments': '{"a":5,"b":7}', 'name': 'Multiply'}, 'type': 'function'}]})

  </TabItem>
</Tabs>

## Defining functions

We'll focus on the [OpenAI function format](https://platform.openai.com/docs/api-reference/chat/create#chat-create-tools) here since as of this writing that is the main model provider that supports function calling. LangChain has a built-in converter that can turn Python functions, Pydantic classes, and LangChain Tools into the OpenAI function format:

### Python function

In [2]:
import json

from langchain_core.utils.function_calling import convert_to_openai_tool


def multiply(a: int, b: int) -> int:
    """Multiply two integers together.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b


print(json.dumps(convert_to_openai_tool(multiply), indent=2))

{
  "type": "function",
  "function": {
    "name": "multiply",
    "description": "Multiply two integers together.",
    "parameters": {
      "type": "object",
      "properties": {
        "a": {
          "type": "integer",
          "description": "First integer"
        },
        "b": {
          "type": "integer",
          "description": "Second integer"
        }
      },
      "required": [
        "a",
        "b"
      ]
    }
  }
}


### Pydantic class

In [3]:
from langchain_core.pydantic_v1 import BaseModel, Field


class multiply(BaseModel):
    """Multiply two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


print(json.dumps(convert_to_openai_tool(multiply), indent=2))

{
  "type": "function",
  "function": {
    "name": "multiply",
    "description": "Multiply two integers together.",
    "parameters": {
      "type": "object",
      "properties": {
        "a": {
          "description": "First integer",
          "type": "integer"
        },
        "b": {
          "description": "Second integer",
          "type": "integer"
        }
      },
      "required": [
        "a",
        "b"
      ]
    }
  }
}


### LangChain Tool

In [6]:
from typing import Any, Type

from langchain_core.tools import BaseTool


class MultiplySchema(BaseModel):
    """Multiply tool schema."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class Multiply(BaseTool):
    args_schema: Type[BaseModel] = MultiplySchema
    name: str = "multiply"
    description: str = "Multiply two integers together."

    def _run(self, a: int, b: int, **kwargs: Any) -> Any:
        return a * b


# Note: we're passing in a Multiply object not the class itself.
print(json.dumps(convert_to_openai_tool(Multiply()), indent=2))

{
  "type": "function",
  "function": {
    "name": "multiply",
    "description": "Multiply two integers together.",
    "parameters": {
      "type": "object",
      "properties": {
        "a": {
          "description": "First integer",
          "type": "integer"
        },
        "b": {
          "description": "Second integer",
          "type": "integer"
        }
      },
      "required": [
        "a",
        "b"
      ]
    }
  }
}


## Binding functions

Now that we've defined a function, we'll want to pass it in to our model.

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")
llm.invoke("what's 5 times three", tools=[convert_to_openai_tool(multiply)])

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_JvOu9oUwMrQHiDekZTbpNCHY', 'function': {'arguments': '{\n  "a": 5,\n  "b": 3\n}', 'name': 'multiply'}, 'type': 'function'}]})

And if we wanted this function to be passed in every time we call the tool, we could bind it to the tool:

In [8]:
llm_with_tool = llm.bind(tools=[convert_to_openai_tool(multiply)])
llm_with_tool.invoke("what's 5 times three")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_cwRoTnD1ux1SnWXLrTj2KlWH', 'function': {'arguments': '{\n  "a": 5,\n  "b": 3\n}', 'name': 'multiply'}, 'type': 'function'}]})

We can also enforce that a tool is called using the [tool_choice](https://platform.openai.com/docs/api-reference/chat/create#chat-create-tools) parameter.

In [10]:
llm_with_tool = llm.bind(
    tools=[convert_to_openai_tool(multiply)],
    tool_choice={"type": "function", "function": {"name": "multiply"}},
)
llm_with_tool.invoke(
    "don't answer my question. no do answer my question. no don't. what's five times four"
)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_sWjLyioSZAtYMQRLMTzncz1v', 'function': {'arguments': '{\n  "a": 5,\n  "b": 4\n}', 'name': 'multiply'}, 'type': 'function'}]})

The [ChatOpenAI](https://api.python.langchain.com/en/latest/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html#langchain_openai.chat_models.base.ChatOpenAI) class even comes with a `bind_tools` helper function that handles converting function-like objects to the OpenAI format and binding them for you:

In [11]:
llm_with_tool = llm.bind_tools([multiply], tool_choice="multiply")
llm_with_tool.invoke("what's 5 times three")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_LCdBa4cbhMJPRdtkhDzpRh7x', 'function': {'arguments': '{\n  "a": 5,\n  "b": 3\n}', 'name': 'multiply'}, 'type': 'function'}]})

## Legacy args `functions` and `function_call`

Until Fall of 2023 the OpenAI API expected arguments `functions` and `funtion_call` instead of `tools` and `tool_choice`, and they had a slightly different format than `tools` and `tool_choice`. LangChain maintains utilities for using the old API if you need to use that as well:

In [13]:
from langchain_core.utils.function_calling import convert_to_openai_function

convert_to_openai_function(multiply)

{'name': 'multiply',
 'description': 'Multiply two integers together.',
 'parameters': {'type': 'object',
  'properties': {'a': {'description': 'First integer', 'type': 'integer'},
   'b': {'description': 'Second integer', 'type': 'integer'}},
  'required': ['a', 'b']}}

In [14]:
llm_with_functions = llm.bind(
    functions=[convert_to_openai_function(multiply)], function_call={"name": "multiply"}
)
llm_with_functions.invoke("what's 3 times a million")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "a": 3,\n  "b": 1000000\n}', 'name': 'multiply'}})

In [15]:
llm_with_functions = llm.bind_functions([multiply], function_call="multiply")
llm_with_functions.invoke("what's 3 times a million")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "a": 3,\n  "b": 1000000\n}', 'name': 'multiply'}})

## Next steps

* **Output parsing**: See [OpenAI Tools output parsers](/docs/modules/model_io/output_parsers/types/openai_tools) and [OpenAI Functions output parsers](/docs/modules/model_io/output_parsers/types/openai_functions) to learn about extracting the function calling API responses into various formats.
* **Tool use**: See how to construct chains and agents that actually call the invoked tools in [these guides](/docs/use_cases/tool_use/).